# Product Recommendation with Feathr on Azure

This notebook demonstrates how Feathr Feature Store can simplify and empower your model training and inference. You will learn:

1. Define sharable features using Feathr API
2. Register features with register API.
3. Create a training dataset via point-in-time feature join with Feathr API
4. Materialize features to online store and then retrieve them with Feathr API

In this tutorial, we use Feathr to create a model that predicts users' product rating. 

## 1. Prerequisite: Use Azure Resource Manager(ARM) to Provision Azure Resources

First step is to provision required cloud resources if you want to use Feathr. Feathr provides a python based client to interact with cloud resources.

Please follow the steps [here](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html) to provision required cloud resources. This will create a new resource group and deploy the needed Azure resources in it. 

If you already have an existing resource group and only want to install few resources manually you can refer to the cli documentation [here](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-cli.html). It provides CLI commands to install the needed resources. 
**Please Note: CLI documentation is for advance users since there are lot of configurations and role assignment that would have to be done manually so it won't work out of box and should just be used for reference. ARM template is the preferred way to deploy.**

The below architecture diagram represents how different resources interact with each other
![Architecture](https://github.com/feathr-ai/feathr/blob/main/docs/images/architecture.png?raw=true)

## 2. Prerequisite: Set the required permissions

Before you proceed further, you would need additional permissions: permission to access the keyvault, permission to access the Storage Blob as a Contributor and permission to submit jobs to Synapse cluster. Run the following lines of command in the [Cloud Shell](https://shell.azure.com) before running the cells below. Please replace the resource_prefix with the prefix you used in ARM template deployment.

```
    resource_prefix="YOUR_RESOURCE_PREFIX"
    synapse_workspace_name="${resource_prefix}syws"
    keyvault_name="${resource_prefix}kv"
    objectId=$(az ad signed-in-user show --query id -o tsv)
    az keyvault update --name $keyvault_name --enable-rbac-authorization false
    az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
    az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
    az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
```


## 3. Prerequisite: Install Feathr and it's dependencies and Login to Azure

Install Feathr and dependencies to run this notebook.

In [ ]:
!pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]"

If you meet errors like 'cannot import FeatherClient from feathr', it may be caused by incompatible version of 'aiohttp'. Please try to install/upgrade it by running the following command:

In [ ]:
!pip install aiohttp==3.8.3

Import Dependencies to make sure everything is installed correctly

In [ ]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
    
import pandas as pd
from pyspark.sql import DataFrame

import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df
from feathr.utils.platform import is_databricks


print(f"Feathr version: {feathr.__version__}")

Login to Azure with a device code (You will see instructions in the output once you execute the cell):

In [ ]:
!az login --use-device-code

In [ ]:
credential = AzureCliCredential(additionally_allowed_tenants=['*'])

If you run into issues where Key vault or other resources are not found through notebook despite being there, make sure you are connected to the right subscription by running the command: 'az account show' and 'az account set --subscription <subscription_id>'

# Feathr Configuration

## Setting the environment variables
Set the environment variables that will be used by Feathr as configuration. Feathr supports configuration via enviroment variables and yaml, you can read more about it [here](https://feathr-ai.github.io/feathr/how-to-guides/feathr-configuration-and-env.html).

**Fill in the `resource_prefix` that you used while provisioning the resources in Step 1 using ARM.**

In [ ]:
# TODO fill the following values
RESOURCE_PREFIX = None           # The prefix value used at the ARM deployment step
AZURE_SYNAPSE_SPARK_POOL = None  # Set Azure Synapse Spark pool name
ADLS_KEY = None                  # Set Azure Data Lake Storage key to use Azure Synapse

PROJECT_NAME = "product_recommendation_synapse_demo"
SPARK_CLUSTER = "azure_synapse"

# TODO if you deployed resources manually using different names, you'll need to change the following values accordingly: 
ADLS_ACCOUNT=f"{RESOURCE_PREFIX}dls"
ADLS_FS_NAME=f"{RESOURCE_PREFIX}fs"
AZURE_SYNAPSE_URL = f"https://{RESOURCE_PREFIX}syws.dev.azuresynapse.net"  # Set Azure Synapse workspace url to use Azure Synapse
KEY_VAULT_URI = f"https://{RESOURCE_PREFIX}kv.vault.azure.net"
REDIS_HOST = f"{RESOURCE_PREFIX}redis.redis.cache.windows.net"
REGISTRY_ENDPOINT = f"https://{RESOURCE_PREFIX}webapp.azurewebsites.net/api/v1"

WORKING_DIR = f"abfss://{ADLS_FS_NAME}@{ADLS_ACCOUNT}.dfs.core.windows.net/{PROJECT_NAME}"


In [ ]:
if "ADLS_KEY" not in os.environ and ADLS_KEY:
    os.environ["ADLS_KEY"] = ADLS_KEY

In [ ]:
if "REDIS_PASSWORD" not in os.environ:
    secret_client = SecretClient(vault_url=KEY_VAULT_URI, credential=credential)
    retrieved_secret = secret_client.get_secret('FEATHR-ONLINE-STORE-CONN').value
    os.environ['REDIS_PASSWORD'] = retrieved_secret.split(",")[1].split("password=", 1)[1]

### Write the configuration as yaml file.

The code below will write this configuration string to a temporary location and load it to Feathr. Please refer to [feathr_config.yaml](https://github.com/feathr-ai/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) for full list of configuration options and details about them.

In [ ]:
config_path = generate_config(
    resource_prefix=RESOURCE_PREFIX,
    project_name=PROJECT_NAME,
    online_store__redis__host=REDIS_HOST,
    feature_registry__api_endpoint=REGISTRY_ENDPOINT,
    spark_config__spark_cluster=SPARK_CLUSTER,
    spark_config__azure_synapse__dev_url=AZURE_SYNAPSE_URL,
    spark_config__azure_synapse__pool_name=AZURE_SYNAPSE_SPARK_POOL,
    spark_config__azure_synapse__workspace_dir=WORKING_DIR,
)

with open(config_path, 'r') as f: 
    print(f.read())

# Define sharable features using Feathr API

In this tutorial, we use Feathr Feature Store and create a model that predicts users' product rating. To make it simple, let's just predict users' rating for ONE product for an e-commerce website. (We have an [advanced demo](../product_recommendation_demo_advanced.ipynb) that predicts ratings for arbitrary products.)


### Initialize Feathr Client

Let's initialize a Feathr client first. The Feathr client provides all the APIs we need to interact with Feathr Feature Store.

In [ ]:
client = FeathrClient(config_path=config_path, credential=credential)

### Understand the Raw Datasets
We have 3 raw datasets to work with: one observation dataset(a.k.a. label dataset) and two raw datasets to generate features.

In [ ]:
# Upload datasets into ADLS
user_observation_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL
)
user_profile_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL
)
user_purchase_history_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL
)

In [ ]:
# Observation dataset(a.k.a. label dataset)
# Observation dataset usually comes with a event_timestamp to denote when the observation happened.
# The label here is product_rating. Our model objective is to predict a user's rating for this product.
pd.read_csv(user_observation_source_path).head()

In [ ]:
# User profile dataset
# Used to generate user features
pd.read_csv(user_profile_source_path).head()

In [ ]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to genearte features.
pd.read_csv(user_purchase_history_source_path).head()

 After a bit of data exploration, we want to create a training dataset like this:

 
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation.jpg?raw=true)

### What's a Feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive. 

In Feathr, feature can be defined by the following characteristics:
1. The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
2. The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
3. The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.

You can feel that this is defined from a feature consumer(a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define a feature as well as how it can be produced, additionally we need:
1. Feature source: what source data that this feature is based on
2. Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html))

### Define Sources Section with Preprocssing
A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) defines where to find the source data and how to use the source data for the upcoming feature transformation. There are different types of feature sources that you can use. HdfsSource is the most commonly used one that can connect you to data lake, Snowflake database tables etc. It's simliar to database connector.

To define HdfsSource, we need:
1. `name`: It's used for you to recognize it. It has to be unique among all other feature source. Here we use `userProfileData`. 
2. `path`: It points to the location that we can find the source data.
3. `preprocessing`(optional): If you want some preprocessing other than provided transformation, you can do it here. This preprocessing will be applied all the transformations of this source.
4. `event_timestamp_column`(optioanl): there are `event_timestamp_column` and `timestamp_format` used for point-in-time join and we will cover them later.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields. 

In [ ]:
def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col

    df = df.withColumn("tax_rate_decimal", col("tax_rate") / 100)
    return df


batch_source = HdfsSource(
    name="userProfileData",
    path=user_profile_source_path,
    preprocessing=feathr_udf_preprocessing,
)

### Define Features on Top of Data Sources
To define features on top of the `HdfsSource`, we need to:
1. specify the key of this feature: feature are like other data, they are keyed by some id. For example, user_id, product_id. You can also define compound keys.
2. specify the name of the feature via `name` parameter and how to transform it from source data via `transform` parameter. Also some other metadata, like `feature_type`.
3. group them together so we know it's from one `HdfsSource` via `FeatureAnchor`. Also give it a unique name via `name` parameter so we can recognize it.

It's called FeatureAnchor since it's like this group of features are anchored to the source. There are other types of features that are computed on top of other features(a.k.a. derived feature which we will cover in next section)

In [ ]:
# Let's define some features for users so our recommendation can be customized for users.
user_id = TypedKey(
    key_column="user_id",
    key_column_type=ValueType.INT32,
    description="user id",
    full_name="product_recommendation.user_id",
)

feature_user_age = Feature(
    name="feature_user_age",
    key=user_id,
    feature_type=INT32,
    transform="age",
)
feature_user_tax_rate = Feature(
    name="feature_user_tax_rate",
    key=user_id,
    feature_type=FLOAT,
    transform="tax_rate_decimal",
)
feature_user_gift_card_balance = Feature(
    name="feature_user_gift_card_balance",
    key=user_id,
    feature_type=FLOAT,
    transform="gift_card_balance",
)
feature_user_has_valid_credit_card = Feature(
    name="feature_user_has_valid_credit_card",
    key=user_id,
    feature_type=BOOLEAN,
    transform="number_of_credit_cards > 0",
)

features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card,
]

user_feature_anchor = FeatureAnchor(
    name="anchored_features", source=batch_source, features=features
)

### Window aggregation features

Using [window aggregations](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) can help us create more powerful features. A window aggregation feature compresses large amount of information into one single feature value. Using our raw data as an example, we have the user's purchase history data that might be quite some rows, we want to create a window aggregation feature that represents their last 90 days of average purchase amount.

To create this window aggregation feature via Feathr, we just need to define the following parameters with `WindowAggTransformation` API:
1. `agg_expr`: the field/column you want to aggregate. It can be a ANSI SQL expression. So we just write `cast_float(purchase_amount)`(the raw data might be in string form, let's cast_float).
2. `agg_func`: the aggregation function you want. We want to use `AVG` here.
3. `window`: the aggregation window size you want. Let's use `90d`. You can tune your windows to create different window aggregation features.

For window aggregation functions, see the supported fields below:

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |

(Note that the `agg_func` should be any of these.)

After you have defined features and sources, bring them together to build an anchor:

In [ ]:
purchase_history_data = HdfsSource(
    name="purchase_history_data",
    path=user_purchase_history_source_path,
    event_timestamp_column="purchase_date",
    timestamp_format="yyyy-MM-dd",
)

agg_features = [
    Feature(
        name="feature_user_avg_purchase_for_90days",
        key=user_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(purchase_amount)", agg_func="AVG", window="90d"
        ),
    )
]

user_agg_feature_anchor = FeatureAnchor(
    name="aggregationFeatures", source=purchase_history_data, features=agg_features
)

### Derived Features Section
Derived features are features that are computed from other Feathr features. They could be computed from anchored features, or other derived features.

Typical usage includes feature cross(f1 * f2), or computing cosine similarity between two features. The syntax works in a similar way.

In [ ]:
feature_user_purchasing_power = DerivedFeature(
    name="feature_user_purchasing_power",
    key=user_id,
    feature_type=FLOAT,
    input_features=[feature_user_gift_card_balance, feature_user_has_valid_credit_card],
    transform="feature_user_gift_card_balance + if(boolean(feature_user_has_valid_credit_card), 100, 0)",
)

### Build Features
Lastly, we need to build these features so that they can be consumed later. Note that we have to build both the "anchor" and the "derived" features.

In [ ]:
client.build_features(
    anchor_list=[user_agg_feature_anchor, user_feature_anchor],
    derived_feature_list=[feature_user_purchasing_power],
)

### Optional: A Special Type of Feature: Request Feature
Sometimes features defined on top of request data(a.k.a. observation data) may have no entity key or timestamp. It is merely a function/transformation executing against request data at runtime.

For example, the day of the week of the request, which is calculated by converting the request UNIX timestamp. In this case, the `source` section should be `INPUT_CONTEXT` to indicate the source of those defined anchors.

We won't cover the details of it in this notebook.

## Create training data using point-in-time correct feature join

A training dataset usually contains `entity id` column(s), multiple `feature` columns, event timestamp column and `label/target` column. 

To create a training dataset using Feathr, we need to provide a feature join settings to specify what features and how these features should be joined to the observation data. 

(To learn more on this topic, please refer to [Point-in-time Correctness](https://feathr-ai.github.io/feathr/concepts/point-in-time-join.html)).

In [ ]:
user_feature_query = FeatureQuery(
    feature_list=[
        "feature_user_age",
        "feature_user_tax_rate",
        "feature_user_gift_card_balance",
        "feature_user_has_valid_credit_card",
        "feature_user_avg_purchase_for_90days",
        "feature_user_purchasing_power",
    ],
    key=user_id,
)

settings = ObservationSettings(
    observation_path=user_observation_source_path,
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd",
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=[user_feature_query],
    output_path=user_profile_source_path.rpartition("/")[0] + f"/product_recommendation_features.avro",
)
client.wait_job_to_finish(timeout_sec=5000)

### Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [ ]:
res_df = get_result_df(client)
res_df.head()

### Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


final_df = (
    res_df
    .drop(["event_timestamp"], axis=1, errors="ignore")
    .fillna(0)
)

X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop(["product_rating"], axis=1),
    final_df["product_rating"].astype("float64"),
    test_size=0.2,
    random_state=42,
)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test.values.flatten(), y_pred))

print(f"Root mean squared error: {rmse}")

## Materialize feature value into offline/online storage

In the previous section, we demonstrated how Feathr can compute feature value to generate training dataset from feature definition on-they-fly.

Now let's talk about how we can use the trained models. We can use the trained models for both online and offline inference. In both cases, we need features to be fed into the models. For offline inference, you can compute and get the features on-demand; or you can store the computed features to some offline database for later offline inference.

For online inference, we can use Feathr to compute and store the features in the online database. Then use it for online inference when the request comes.

![img](../../images/online_inference.jpg)


In this section, we will focus on materialize features to online store. For materialization to offline store, you can check out our [user guide](https://feathr-ai.github.io/feathr/concepts/materializing-features.html#materializing-features-to-offline-store).

We can push the computed features to the online store(Redis) like below:

In [ ]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings(
    name="user_feature_setting",
    sinks=[redisSink],
    feature_names=["feature_user_age", "feature_user_gift_card_balance"],
)

client.materialize_features(settings=settings, allow_materialize_non_agg_feature=True)
client.wait_job_to_finish(timeout_sec=5000)

### Fetch feature value from online store
We can then get the features from the online store (Redis) via the client's `get_online_features` or `multi_get_online_features` API.

In [ ]:
client.get_online_features(
    "user_features", "2", ["feature_user_age", "feature_user_gift_card_balance"]
)

In [ ]:
client.multi_get_online_features(
    "user_features", ["1", "2"], ["feature_user_age", "feature_user_gift_card_balance"]
)

### Registering and Fetching features

We can also register the features and share them across teams:

In [ ]:
try:
    client.register_features()
except KeyError:
    # TODO temporarily go around the "Already exists" error
    pass
print(client.list_registered_features(project_name=PROJECT_NAME))

## Summary
In this notebook you learnt how to set up Feathr and use it to create features, register features and use those features for model training and inferencing.

We hope this example gave you a good sense of Feathr's capabilities and how you could leverage it within your organization's MLOps workflow.